In [1]:
import sys
import pandas as pd
from orderbook_test import OrderBook
import glob

In [2]:
book1 = OrderBook()

In [3]:
df = pd.read_parquet('MBO.parquet')

In [4]:
df.head()

,ts_event,rtype,publisher_id,instrument_id,action,side,price,size,channel_id,order_id,flags,ts_in_delta,sequence,symbol
ts_recv,,,,,,,,,,,,,,
2024-01-01 00:00:00+00:00,2023-12-31 13:00:04.466568089+00:00,160,1,38601,R,N,NaN,0,26,0,40,0,0,CLF5
2024-01-01 00:00:00+00:00,2023-12-31 13:00:04.466568089+00:00,160,1,38601,A,B,68.0,1,26,8048898538378,40,0,50447,CLF5
2024-01-01 00:00:00+00:00,2023-12-31 13:00:04.466568089+00:00,160,1,38601,A,B,67.0,1,26,8048898527545,40,0,50447,CLF5
2024-01-01 00:00:00+00:00,2023-12-31 13:00:04.466568089+00:00,160,1,38601,A,B,66.5,1,26,8048898560786,40,0,50447,CLF5
2024-01-01 00:00:00+00:00,2023-12-31 13:00:04.466568089+00:00,160,1,38601,A,B,66.0,1,26,8048898569034,40,0,50447,CLF5


In [5]:
def dispatch(action, oid, side, price, size):
    if action == "Add":
        book1.addOrder(oid, side, price, size)
    elif action == "Cancel":
        book1.cancelOrder(oid)
    elif action == "Modify":
        book1.modifyOrder(oid, size)
    elif action == "Fill":
        book1.fillOrder(oid, size)

In [6]:
# for row in df:
#     action = row.action
#     order_id = row.order_id
#     side = row.side
#     price = row.price
#     size = row.size 
#     dispatch(action, order_id, side, price, size)
required = {"action", "order_id", "side", "price", "size"}
missing = required.difference(df.columns)
if missing:
    raise ValueError(f"Missing required columns: {sorted(missing)}")

for row in df.itertuples(index=False):
    act  = row.action        # 'A','C','M','F' or 'R'
    if act not in ("A", "C", "M", "F", "Add", "Cancel", "Modify", "Fill"):
        continue  # skip refresh/unknown rows (e.g., 'R')

    oid  = str(row.order_id)
    side = row.side        
    price= row.price
    size = row.size

    # deal with NaNs sometimes present in refresh rows
    if pd.isna(price) or pd.isna(size):
        continue

    dispatch(act, oid, side, price, size)

In [7]:
# ---------- After processing: print top-of-book + global stats ----------

def _peek_best(sd, want_max=False):
    """Return (price, level) or None if empty."""
    if not sd:
        return None
    try:
        return sd.peekitem(-1 if want_max else 0)  # SortedDict
    except AttributeError:
        k = max(sd) if want_max else min(sd)
        return k, sd[k]

def _sum_levels(sd):
    total_size = 0
    total_count = 0
    for level in sd.values():
        total_size += getattr(level, "total_size", 0)
        total_count += getattr(level, "order_count", 0)
    return total_size, total_count

def print_top_and_stats(book1):
    bids = getattr(book1, "bids", None)
    asks = getattr(book1, "asks", None)

    best_bid = _peek_best(bids, want_max=True)
    best_ask = _peek_best(asks, want_max=False)

    def fmt(pair, name):
        if pair is None:
            return f"{name}: (empty)"
        px, lvl = pair
        return f"{name}: {px:.4f} x {lvl.total_size} (orders={lvl.order_count})"

    # global stats
    if hasattr(book1, "bid_stats") and hasattr(book1.bid_stats, "total_size"):
        bid_total_size = book1.bid_stats.total_size
        bid_total_count = getattr(book1.bid_stats, "total_count", None)
    else:
        bid_total_size, bid_total_count = _sum_levels(bids or {})

    if hasattr(book1, "ask_stats") and hasattr(book1.ask_stats, "total_size"):
        ask_total_size = book1.ask_stats.total_size
        ask_total_count = getattr(book1.ask_stats, "total_count", None)
    else:
        ask_total_size, ask_total_count = _sum_levels(asks or {})

    print("\n=== TOP OF BOOK ===")
    print(fmt(best_bid, "Best Bid"))
    print(fmt(best_ask, "Best Ask"))

    print("\n=== GLOBAL STATS ===")
    print(f"Bids: total_size={bid_total_size}, total_orders={bid_total_count}")
    print(f"Asks: total_size={ask_total_size}, total_orders={ask_total_count}")

# call this at the very end
print_top_and_stats(book1)


=== TOP OF BOOK ===
Best Bid: (empty)
Best Ask: (empty)

=== GLOBAL STATS ===
Bids: total_size=0, total_orders=0
Asks: total_size=0, total_orders=0
